Apply  Pandas and MongoDB skills to solve real-world data cleaning and wrangling problems for handling patient data.

The dataset for this contains a MongoDB dump and two CSV files which will be used for the tasks in this notebook. More details about these can be found below.

The task here is to collate data coming from three different sources and further perform analysis on data regarding COVID-19 patients.

In [2]:
#1. import
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np
import datetime

In [3]:
#2 #3
client=MongoClient()
client.list_database_names()
patientdb=client['patientdb']
patientdb.list_collection_names()



['conditions', 'patients']

In [14]:
#4 Import the ‘patient_data_1.csv’ and ‘patient_data_2.csv’ 
patient1=pd.read_csv('patient_data_1.csv')
patient2=pd.read_csv('patient_data_2.csv')
#Harmonize patient1
patient1['GENDER'] = patient1['GENDER'].replace({'Male': 'M', 'Female': 'F'})
patient1 = patient1.rename(columns={'Id': '_id'})
patient1.head()

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P004753,2012-01-12,NaN,asian,hispanic,F,Westford,Middlesex County,Massachusetts
1,P009495,1961-12-01,M,white,nonhispanic,F,Seekonk,Bristol County,Massachusetts
2,P010851,1997-10-21,NaN,black,nonhispanic,M,Quincy,Norfolk County,Massachusetts
3,P009020,1975-08-15,M,white,nonhispanic,F,Worcester,Worcester County,Massachusetts
4,P009779,2020-02-25,NaN,white,nonhispanic,F,Bolton,Worcester County,Massachusetts


In [15]:
# Harmonize patient2
patient2.columns = patient2.columns.str.upper()
patient2['ETHNICITY'] = patient2['ETHNICITY'].replace({'non-hispanic': 'nonhispanic'})
patient2 = patient2.rename(columns={'ID': '_id'})
patient2.head()

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P005102,1969-05-11,M,white,nonhispanic,F,East Longmeadow,Hampden County,Massachusetts
1,P001814,1957-12-31,M,white,nonhispanic,M,Somerville,Middlesex County,Massachusetts
2,P000373,1989-07-20,S,white,nonhispanic,F,Cambridge,Middlesex County,Massachusetts
3,P007365,1967-10-26,M,white,nonhispanic,F,Mashpee,Barnstable County,Massachusetts
4,P010311,1965-02-14,M,white,nonhispanic,F,Somerville,Middlesex County,Massachusetts


In [37]:
#5. From the ‘patients collection, retrieve the patients who are alive.
patients=patientdb['patients']
alive= list(patients.find({'DEATHDATE':{'$exists':False}}))
print(patients.count_documents({'DEATHDATE':{'$exists':False}}))
df_alive=pd.DataFrame(alive)
df_alive

8700


,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P000001,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts
1,P000002,2016-08-01,NaN,white,nonhispanic,F,Walpole,Norfolk County,Massachusetts
2,P000003,1992-06-30,S,white,nonhispanic,M,Chicopee,Hampden County,Massachusetts
3,P000004,2004-01-09,NaN,white,nonhispanic,F,Pembroke,Plymouth County,Massachusetts
4,P000005,1996-11-15,NaN,white,nonhispanic,M,Boston,Suffolk County,Massachusetts
...,...,...,...,...,...,...,...,...,...
8695,P012343,1960-01-05,M,white,nonhispanic,F,Dennis,Barnstable County,Massachusetts
8696,P012345,1980-11-12,M,white,nonhispanic,F,Malden,Middlesex County,Massachusetts
8697,P012347,1960-11-10,M,black,nonhispanic,F,North Andover,Essex County,Massachusetts
8698,P012351,1918-10-20,S,white,nonhispanic,F,Waltham,Middlesex County,Massachusetts


In [40]:
#6.Concatenate the above three DataFrames.
df3=pd.concat([patient1,patient2,df_alive])
df3=df3.sort_values('_id')
df3

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE
0,P000001,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts
1,P000002,2016-08-01,NaN,white,nonhispanic,F,Walpole,Norfolk County,Massachusetts
2,P000003,1992-06-30,S,white,nonhispanic,M,Chicopee,Hampden County,Massachusetts
3,P000004,2004-01-09,NaN,white,nonhispanic,F,Pembroke,Plymouth County,Massachusetts
4,P000005,1996-11-15,NaN,white,nonhispanic,M,Boston,Suffolk County,Massachusetts
...,...,...,...,...,...,...,...,...,...
591,P012346,1990-08-25,M,white,nonhispanic,F,Dennis,Barnstable County,Massachusetts
8697,P012347,1960-11-10,M,black,nonhispanic,F,North Andover,Essex County,Massachusetts
249,P012348,1962-08-17,M,asian,nonhispanic,F,Wellesley,Norfolk County,Massachusetts
8698,P012351,1918-10-20,S,white,nonhispanic,F,Waltham,Middlesex County,Massachusetts


In [63]:
#7.Compute the current age of the patients and store them in another column called ‘AGE’.
df3['BIRTHDATE'] = pd.to_datetime(df3['BIRTHDATE'])
df3['AGE'] = (datetime.datetime.now() - df3['BIRTHDATE']).dt.days // 365
df3

,_id,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,COUNTY,STATE,AGE
0,P000001,2017-08-24,NaN,white,nonhispanic,M,Springfield,Hampden County,Massachusetts,6
1,P000002,2016-08-01,NaN,white,nonhispanic,F,Walpole,Norfolk County,Massachusetts,7
2,P000003,1992-06-30,S,white,nonhispanic,M,Chicopee,Hampden County,Massachusetts,31
3,P000004,2004-01-09,NaN,white,nonhispanic,F,Pembroke,Plymouth County,Massachusetts,20
4,P000005,1996-11-15,NaN,white,nonhispanic,M,Boston,Suffolk County,Massachusetts,27
...,...,...,...,...,...,...,...,...,...,...
591,P012346,1990-08-25,M,white,nonhispanic,F,Dennis,Barnstable County,Massachusetts,33
8697,P012347,1960-11-10,M,black,nonhispanic,F,North Andover,Essex County,Massachusetts,63
249,P012348,1962-08-17,M,asian,nonhispanic,F,Wellesley,Norfolk County,Massachusetts,61
8698,P012351,1918-10-20,S,white,nonhispanic,F,Waltham,Middlesex County,Massachusetts,105


In [72]:
#8. Insert the above DataFrame to a collection named ‘patients_clean’. 
patientdb.patients_clean.insert_many(df3.to_dict('records'))
print(patientdb.list_collection_names())
patientdb['patients_clean'].count()



['conditions', 'patients_clean', 'patients']


/var/folders/tn/1rpmlfx112j95v9ws4cxqb8c0000gn/T/ipykernel_13953/303061839.py:4: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  patientdb['patients_clean'].count()


10000

In [168]:
#9.obtain the ID’s of patients that were diagnosed with COVID-19.
condition=patientdb['conditions']
covidid=condition.find({'conditions':'COVID-19'},{'_id':1})
covidresult=list(covidid)
print(len(covidresult))
print(covidresult[0:10])


8820
[{'_id': 'E000003'}, {'_id': 'E000005'}, {'_id': 'E000009'}, {'_id': 'E000010'}, {'_id': 'E000011'}, {'_id': 'E000012'}, {'_id': 'E000015'}, {'_id': 'E000019'}, {'_id': 'E000024'}, {'_id': 'E000028'}]


In [161]:
#10.obtain COVID-19 patients who also had ‘Cough (finding)’ and ‘Fever (finding)’ but did not have ‘Loss of taste (finding)’
fever=condition.aggregate (
    [
        {
            '$match': {
            'conditions': {
                '$all': ['COVID-19', 'Cough (finding)', 'Fever (finding)'],
                '$nin': ['Loss of taste (finding)']
        }
            }}
    ]
)
result=list(fever)
print(len(result))
result[0:5]

2613


[{'_id': 'E000005',
  'patient': 'P000002',
  'conditions': ['Cough (finding)',
   'Sputum finding (finding)',
   'Diarrhea symptom (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19']},
 {'_id': 'E000010',
  'patient': 'P000004',
  'conditions': ['Cough (finding)',
   'Sputum finding (finding)',
   'Nausea (finding)',
   'Vomiting symptom (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19']},
 {'_id': 'E000011',
  'patient': 'P000005',
  'conditions': ['Nasal congestion (finding)',
   'Cough (finding)',
   'Sputum finding (finding)',
   'Fatigue (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19']},
 {'_id': 'E000028',
  'patient': 'P000011',
  'conditions': ['Cough (finding)',
   'Fatigue (finding)',
   'Dyspnea (finding)',
   'Wheezing (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19']},
 {'_id': 'E000036',
  'patient': 'P000013',
  'conditions': ['Cough (finding)',
   'Sputum finding (finding)',
 

In [164]:
#11.Remove dead patients here.
clean=patientdb['patients_clean']

a=condition.aggregate([{
    '$match':
    {'conditions':'COVID-19'}
},{
    '$lookup':{
        'from':'patients_clean',
        'localField': 'patient',
        'foreignField':'_id',
        'as': 'patients_info'
    }
},{
'$match': {'patients_info':{'$ne':[]}}
}])

result11=list(a)
print(len(result11))
result11[0:5]


8463


[{'_id': 'E000003',
  'patient': 'P000001',
  'conditions': ['Fever (finding)', 'Suspected COVID-19', 'COVID-19'],
  'patients_info': [{'_id': 'P000001',
    'BIRTHDATE': datetime.datetime(2017, 8, 24, 0, 0),
    'MARITAL': nan,
    'RACE': 'white',
    'ETHNICITY': 'nonhispanic',
    'GENDER': 'M',
    'CITY': 'Springfield',
    'COUNTY': 'Hampden County',
    'STATE': 'Massachusetts',
    'AGE': 6}]},
 {'_id': 'E000005',
  'patient': 'P000002',
  'conditions': ['Cough (finding)',
   'Sputum finding (finding)',
   'Diarrhea symptom (finding)',
   'Fever (finding)',
   'Suspected COVID-19',
   'COVID-19'],
  'patients_info': [{'_id': 'P000002',
    'BIRTHDATE': datetime.datetime(2016, 8, 1, 0, 0),
    'MARITAL': nan,
    'RACE': 'white',
    'ETHNICITY': 'nonhispanic',
    'GENDER': 'F',
    'CITY': 'Walpole',
    'COUNTY': 'Norfolk County',
    'STATE': 'Massachusetts',
    'AGE': 7}]},
 {'_id': 'E000009',
  'patient': 'P000003',
  'conditions': ['Headache (finding)',
   'Cough (findi

In [156]:
#12.calculate the total number of alive COVID-19 patients for each race.
b=condition.aggregate([{
    '$match':
    {'conditions':'COVID-19'}
},{
    '$lookup':{
        'from':'patients_clean',
        'localField': 'patient',
        'foreignField':'_id',
        'as': 'patients_info'
    }
},{
'$match': {'patients_info':{'$ne':[]}}
},{
    '$group':{
    '_id':'$patients_info.RACE',
    'numPatient':{'$sum':1}
    }}
])
list(b)

[{'_id': ['other'], 'numPatient': 8},
 {'_id': ['black'], 'numPatient': 709},
 {'_id': ['native'], 'numPatient': 47},
 {'_id': ['asian'], 'numPatient': 610},
 {'_id': ['white'], 'numPatient': 7089}]